imports + config

In [ ]:
import subprocess
import itertools
import re
import pandas as pd
from pathlib import Path
from IPython.display import display

SCRIPT_PATH = str(Path("block2_group4.py").resolve())
PYTHON_BIN = "python"

hyperparameter grids

In [ ]:
train_epochs_list    = [10.0, 11.0, 12.0]
lr_list              = [10.0, 11.5, 13.0]
weight_decay_list    = [0.0153, 0.01]
label_smoothing_list = [0.2, 0.15, 0.1]

n_runs_coarse = 5   # for quick coarse search
n_runs_fine   = 25  # for rerunning the best configs later

run_experiment helper

In [ ]:
def run_experiment(train_epochs, lr, weight_decay, label_smoothing, n_runs, exp_name=None):
    if exp_name is None:
        exp_name = f"te{train_epochs}_lr{lr}_wd{weight_decay}_ls{label_smoothing}"

    cmd = [
        PYTHON_BIN, SCRIPT_PATH,
        "--train_epochs", str(train_epochs),
        "--lr", str(lr),
        "--weight_decay", str(weight_decay),
        "--label_smoothing", str(label_smoothing),
        "--n_runs", str(n_runs),
        "--exp_name", exp_name,
    ]

    print("Running:", " ".join(cmd))

    out = subprocess.check_output(cmd, text=True)

    m_acc = re.search(r"Mean accuracy:\s+([0-9.]+)\s+Std:\s+([0-9.]+)", out)
    m_time = re.search(r"Mean training time \(s\):\s+([0-9.]+)\s+Std:\s+([0-9.]+)", out)

    mean_acc  = float(m_acc.group(1))  if m_acc  else None
    std_acc   = float(m_acc.group(2))  if m_acc  else None
    mean_time = float(m_time.group(1)) if m_time else None
    std_time  = float(m_time.group(2)) if m_time else None

    return {
        "train_epochs": train_epochs,
        "lr": lr,
        "weight_decay": weight_decay,
        "label_smoothing": label_smoothing,
        "n_runs": n_runs,
        "exp_name": exp_name,
        "mean_acc": mean_acc,
        "std_acc": std_acc,
        "mean_time": mean_time,
        "std_time": std_time,
    }


coarse sweep

In [ ]:
coarse_results = []
for train_epochs, lr, weight_decay, label_smoothing in itertools.product(
    train_epochs_list, lr_list, weight_decay_list, label_smoothing_list
):
    res = run_experiment(
        train_epochs=train_epochs,
        lr=lr,
        weight_decay=weight_decay,
        label_smoothing=label_smoothing,
        n_runs=n_runs_coarse,
    )
    coarse_results.append(res)

results_coarse = pd.DataFrame(coarse_results)
display(results_coarse.sort_values("mean_acc", ascending=False))

print("\nTop configs with mean_time < 3.2s:")
display(
    results_coarse[results_coarse["mean_time"] < 3.2]
    .sort_values("mean_acc", ascending=False)
    .head(10)
)


fine sweep of best configs

In [ ]:
time_budget = 3.2  # seconds

candidate_rows = (
    results_coarse[results_coarse["mean_time"] <= time_budget]
    .sort_values("mean_acc", ascending=False)
    .head(5)
)

fine_results = []
for row in candidate_rows.itertuples():
    res = run_experiment(
        train_epochs=row.train_epochs,
        lr=row.lr,
        weight_decay=row.weight_decay,
        label_smoothing=row.label_smoothing,
        n_runs=n_runs_fine,
        exp_name=f"{row.exp_name}_fine",
    )
    fine_results.append(res)

results_fine = pd.DataFrame(fine_results)
display(results_fine.sort_values("mean_acc", ascending=False))

results_coarse.to_csv("coarse_sweep_results.csv", index=False)
results_fine.to_csv("fine_sweep_results.csv", index=False)
